### C# Interactive Notebook - Carbon Example No.3

This example is a slightly more spohisticated version of C# example notebook no.2. A cross-tabulation report of brand popularity by month over a 5 year period is converted into a Microsoft DataFrame which is input to [Plotly.net](https://plotly.net/reference/index.html) to create a line chart.

In [1]:
#r "nuget: RCS.Carbon.Tables"
#r "nuget: RCS.Carbon.Licensing.RedCentre"
#r "nuget: DataView.InteractiveExtension,*-*"
#r "nuget: Microsoft.ML.DataView"
#r "nuget: Microsoft.Data.Analysis"
#r "nuget: Plotly.NET"
#r "nuget: Plotly.NET.Interactive"
#r "nuget: FSharp.Data"

using RCS.Carbon.Tables;
using RCS.Carbon.Variables;
using RCS.Carbon.Shared;
using RCS.Carbon.Licensing.Shared;
using RCS.Carbon.Licensing.RedCentre;

Installed Packages DataView.InteractiveExtension, 1.0.142 FSharp.Data, 6.4.1 Microsoft.Data.Analysis, 0.22.2 Microsoft.ML.DataView, 4.0.2 Plotly.NET, 5.1.0 Plotly.NET.Interactive, 5.0.0 RCS.Carbon.Licensing.RedCentre, 9.1.10 RCS.Carbon.Tables, 9.1.10

Loading extension script from `C:\Users\greg\.nuget\packages\dataview.interactiveextension\1.0.142\interactive-extensions\dotnet\extension.dib`

Added support IDataView to kernel .NET.

Loading extensions from `C:\Users\greg\.nuget\packages\plotly.net.interactive\5.0.0\lib\netstandard2.1\Plotly.NET.Interactive.dll`

Loading extensions from `C:\Users\greg\.nuget\packages\microsoft.data.analysis\0.22.2\interactive-extensions\dotnet\Microsoft.Data.Analysis.Interactive.dll`

### Generate the cross-tabulation report

- The `LoginId` parameters are the Id and password of the Carbon guest user account which has sufficient data quota to process the large number of cases used in the example.
- Note that the report output has to be tweaked slightly to be in the exact format required by DataFrames. The X axis name is prefixed so that the X and Y axis labels border the rectangular data. The values '*' are converted to '0' because DataFrames expect numeric values.

In [2]:

var provider = new RedCentreLicensingProvider();
var engine = new CrossTabEngine(provider);
await engine.LoginId("16499372", "C6H12O6");
engine.OpenJob("rcsruby", "demo");

var sprops  = new XSpecProperties(); 
sprops.InitAsMissing = true;

var dprops  = new XDisplayProperties();
dprops.Output.Format = XOutputFormat.TSV;
dprops.Cells.Frequencies.Visible = false;
dprops.Cells.RowPercents.Visible = true;    // Need this as Carbon table is transposed.
dprops.Cells.ColumnPercents.Visible = false;
dprops.Decimals.Frequencies = 0;
dprops.Decimals.Expressions = 2;
dprops.Decimals.Statistics = 2;

dprops.Titles.Top.Visible = false;
dprops.Titles.Side.Visible = false;
dprops.Titles.Filter.Visible = false;
dprops.Titles.Weight.Visible = false;
dprops.Titles.Status.Visible = false;
dprops.Titles.Name.Visible = false;

dprops.Corner.Priority = XPriority.Top;

string chartXaxis ="month";
string seriesPlots = "uba";

string ret = engine.GenTab("Example 03", seriesPlots, chartXaxis + "(*)", null, null, sprops, dprops);

string strRet = ret.Replace('*','0');

Console.WriteLine(chartXaxis + strRet);

Error: System.ApplicationException: User 'CarbonGuest' Id '16499372' licence has been borrowed by a different computer
   at RCS.Licensing.Stdlib.LicensingClient.BorrowIdAuthenticate(BorrowIdAuthenticateRequest request)
   at RCS.Licensing.Stdlib.LicensingClient.BorrowIdAuthenticate(String id, String password)
   at RCS.Carbon.Licensing.RedCentre.RedCentreLicensingProvider.LoginId(String userId, String password, Boolean skipCache)
   at RCS.Carbon.Variables.VEngine.LoginId(String userId, String password, Boolean skipCache)
   at Submission#4.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

### Convert Carbon data to a DataFrame

Also creates a list of chart series names

In [3]:
using Microsoft.Data.Analysis;
using Microsoft.ML;
using System.Collections.Generic;

var df1 = DataFrame.LoadCsvFromString(chartXaxis +  strRet, '\t');

var chartSeriesNames = df1.Columns.Skip(1).Select(c => c.Name).ToArray();   // Skip first 1 which is the label "month"
chartSeriesNames

Error: System.FormatException: Empty file
   at Microsoft.Data.Analysis.DataFrame.ReadCsvLinesIntoDataFrame(WrappedStreamReaderOrStringReader wrappedReader, Char separator, Boolean header, String[] columnNames, Type[] dataTypes, Int64 numberOfRowsToRead, Int32 guessRows, Boolean addIndexColumn, Boolean renameDuplicatedColumns, CultureInfo cultureInfo, Func`2 guessTypeFunction)
   at Microsoft.Data.Analysis.DataFrame.LoadCsvFromString(String csvString, Char separator, Boolean header, String[] columnNames, Type[] dataTypes, Int64 numberOfRowsToRead, Int32 guessRows, Boolean addIndexColumn, Boolean renameDuplicatedColumns, CultureInfo cultureInfo, Func`2 guessTypeFunction)
   at Submission#5.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, Cancellat

### Plotly.Net

API reference is https://plotly.net/reference/index.html

This produces an interactive inline chart. Click legend to toggle series on and off. Use Zoom and Pan to see detail.

In [4]:
using Plotly.NET;
using Plotly.NET.Interactive;
using Plotly.NET.LayoutObjects;
using Microsoft.FSharp.Core;
using Microsoft.FSharp.Collections;

In [5]:

LinearAxis catAxis = new LinearAxis();
catAxis.SetValue("axistype","Category");
catAxis.SetValue("title", chartXaxis);
catAxis.SetValue("zerolinecolor", "#ffff");
catAxis.SetValue("gridcolor", "#ffff");
catAxis.SetValue("showline", true);
catAxis.SetValue("zerolinewidth",2);

In [6]:

LinearAxis yAxis = new LinearAxis();
yAxis.SetValue("title", "Respondents");
yAxis.SetValue("zerolinecolor", "#ffff");
yAxis.SetValue("gridcolor", "#ffff");
yAxis.SetValue("showline", true);
yAxis.SetValue("zerolinewidth",2);

Layout layout = new Layout();
layout.SetValue("xaxis", catAxis);
layout.SetValue("yaxis", yAxis);
layout.SetValue("title", "Data from Carbon");
layout.SetValue("plot_bgcolor", "#e5ecf6");
layout.SetValue("showlegend", true);

var traces = new List<Trace>();
for (int a = 0; a < chartSeriesNames.Length; a++)
{
    Trace t =new Trace("line");
    t.SetValue("x", df1[chartXaxis]); 
    t.SetValue("y", df1[chartSeriesNames[a]]); 
    t.SetValue("name",chartSeriesNames[a]); 
    traces.Add(t);
}

var fig = GenericChart.Figure.create(ListModule.OfSeq(traces),layout);
GenericChart.fromFigure(fig);

Error: (26,24): error CS0117: 'GenericChart' does not contain a definition for 'Figure'

In [7]:
bool closed = engine.CloseJob();
Console.WriteLine($"Job closed = {closed} at {DateTime.Now:HH:mm:ss}");
int count = await engine.LogoutId("16499372");
Console.WriteLine($"Logout done. Licence borrow count is down to {count}");

Job closed = False at 11:10:46
Logout done. Licence borrow count is down to 1
